# Распределенное обучение классических моделей

Поговорим про то, как решать задачу машинного обучения, когда самих наблюдений очень много и они все не помещаются на машину.

Центральная идея во всех алгоритмах - параллельно на нескольких машинах посчитать частичные элементы, которые требуются для принятия решения, передать их на центральную машину и сделать шаг алгоритма.

Для обучения линейных моделей на различных машинах будем считать градиент и на главной машине делать шаг градиентного спуска.

Для деревьев решений на различных машинах будем считать распределение по корзинкам (по бинам) и на главной машине будет определять порог для определенного признака.

### Распределенное обучение VW

Vowpal Wabbit также умеет работать распределенно, что делает его универсальным инструментом для обучения линейных моделей на больших данных. Для работы он использует дополнительный компонент - `spanning_tree` - это специальный процесс, который координирует работу различных воркеров между собой.

Про него можно также думать, как про корневую вершину в алгоритме "Tree Allreduce", который используется для эффективной утилизации сети при обучении.

Чтобы иметь возможность использовать `spanning_tree`, необходимо собрать VW руками.


Собирем VW. Делать это нужно с суперпользоателя, поэтому удобнее всего запускать из терминала.

```bash
apt update && \
apt install git psmisc -y && \
apt install libboost-dev libboost-program-options-dev libboost-system-dev libboost-thread-dev libboost-math-dev libboost-test-dev zlib1g-dev cmake g++ -y 


wget https://github.com/google/flatbuffers/archive/v1.12.0.tar.gz && \
tar -xzf v1.12.0.tar.gz && \
cd flatbuffers-1.12.0 && \
mkdir build_dir && \
cd build_dir && \
cmake -G "Unix Makefiles" -DFLATBUFFERS_BUILD_TESTS=Off -DFLATBUFFERS_INSTALL=On -DCMAKE_BUILD_TYPE=Release DFLATBUFFERS_BUILD_FLATHASH=Off .. && \
make install -j$(nproc) && \
cd ../..

git clone --recursive https://github.com/VowpalWabbit/vowpal_wabbit.git && \
cd vowpal_wabbit && \
sudo make && \
cd build && \
sudo make install -j$(nproc)
```

**Хозяйке на заметку** Чтобы получить рутовый доступ с кластера в Azure через Jupyter можно открыть терминал и по ssh подключиться к пользователю `azureuser`. Текущий пользователь `spark` к сожалению имеет очень мало прав.

```bash
ssh azureuser@localhost
sudo su
```

In [1]:
%%local

! which spanning_tree

In [2]:
%%local

! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
! unzip drugsCom_raw.zip

--2021-02-22 08:20:20--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42989872 (41M) [application/x-httpd-php]
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip    100%[===================>]  41.00M  67.3MB/s    in 0.6s    

2021-02-22 08:20:20 (67.3 MB/s) - ‘drugsCom_raw.zip’ saved [42989872/42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [8]:
%%local

! hdfs dfs -rm -r /drugs/data || true
! hdfs dfs -mkdir -p /drugs/data

21/02/22 08:28:29 WARN azure.AzureFileSystemThreadPoolExecutor: Disabling threads for Delete operation as thread count 0 is <= 1
21/02/22 08:28:29 INFO azure.AzureFileSystemThreadPoolExecutor: Time taken for Delete operation is: 25 ms with threads: 0
Deleted /drugs/data
/bin/sh: -c: line 0: syntax error near unexpected token `('
/bin/sh: -c: line 0: ` cat drugsComTrain_raw.tsv <(tail -n +2 drugsComTest_raw.tsv) | hdfs dfs -put - /drugs/data/drugs.tsv'


Выгрузим датасет с препаратами.

In [9]:
%%bash

cat drugsComTrain_raw.tsv <(tail -n +2 drugsComTest_raw.tsv) | hdfs dfs -put - /drugs/data/drugs.tsv

In [11]:
%%local

! hdfs dfs -ls  -h /drugs/data

Found 1 items
-rw-r--r--   1 spark supergroup    107.2 M 2021-02-22 08:29 /drugs/data/drugs.tsv


In [4]:
%%configure -f
{"executorMemory": "20000M", "executorCores": 4, "numExecutors":5}

In [57]:
from pyspark.sql import functions as F
from datetime import datetime
import re

In [5]:
sc = spark.sparkContext

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1613979686616_0007,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

In [12]:
data = spark.read.option("delimiter", "\t").csv('/drugs/data/*', header=True, inferSchema=True)

In [13]:
data.show()

+--------------------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|                 _c0|            drugName|           condition|              review|rating|             date|usefulCount|
+--------------------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|              206461|           Valsartan|Left Ventricular ...|"""It has no side...|   9.0|     May 20, 2012|         27|
|               95260|          Guanfacine|                ADHD|"""My son is half...|  null|             null|       null|
|We have tried man...|                 8.0|      April 27, 2010|                 192|  null|             null|       null|
|               92703|              Lybrel|       Birth Control|"""I used to take...|  null|             null|       null|
|The positive side...|                 5.0|   December 14, 2009|                  17|  null|             null|       null|
|              1

Мы будем запускать 2 воркера. Поэтмоу разделим весь датасет на 3 части - 2 равные для воркером и 1 маленькую часть для теста.

In [45]:
part1, part2, test = (
    data
    .na.drop('any')
    .randomSplit([0.45, 0.45, 0.1], 422)
)

Соберем датасет на спарке

In [46]:
def convert_to_vw(data):
    target = data['usefulCount']
    
    drug_name = data['drugName'].lower().replace(' ', '_')
    condition = data['condition'].lower().replace(' ', '_')
    
    raw_text = data['review'].lower()
    word_pattern = re.compile(r"[a-zA-Z0-9_]+")
    words = [match.group(0) for match in re.finditer(word_pattern, raw_text)]
    review = ' '.join(words)
    
    rating = data['rating']
    
    weekday = datetime.strptime(data['date'], '%B %d, %Y').weekday()
    
    template = "{target} |d {drug_name} |c {condition} |r {review} |w {weekday} |s rating:{rating}"
    return template.format(
        target=target,
        drug_name=drug_name,
        condition=condition,
        review=review,
        weekday=weekday,
        rating=rating
    )

In [48]:
%%local

! hdfs dfs -rm -r /drugs/*.vw

21/02/22 09:32:58 WARN azure.AzureFileSystemThreadPoolExecutor: Disabling threads for Delete operation as thread count 0 is <= 1
21/02/22 09:32:58 WARN azure.NativeAzureFileSystem: Attempt to delete non-existent directory drugs/part1.vw/_temporary
21/02/22 09:32:58 INFO azure.AzureFileSystemThreadPoolExecutor: Time taken for Delete operation is: 32 ms with threads: 0
Deleted /drugs/part1.vw


In [49]:
part1.rdd.map(convert_to_vw).saveAsTextFile('/drugs/part1.vw')
part2.rdd.map(convert_to_vw).saveAsTextFile('/drugs/part2.vw')
test.rdd.map(convert_to_vw).saveAsTextFile('/drugs/test.vw')

In [53]:
%%local

! hdfs dfs -cat /drugs/part1.vw/* > train.part1.vw
! hdfs dfs -cat /drugs/part2.vw/* > train.part2.vw
! hdfs dfs -cat /drugs/test.vw/* > test.vw

Посмотрим, какие результаты мы получим, если просто запустим VW на всем файле.

In [72]:
%%local

! cat train.*.vw > train.full.vw

In [67]:
%%local

import numpy as np
from sklearn.metrics import r2_score


def calc_r2(predictions_filename, answers_filename):
    def read_target_from_vw(vw_record):
        return float(vw_record.split(' ')[0])
    
    with open(predictions_filename, 'r') as f:
        y_pred = np.array([float(value) for value in f.readlines()])
        
    with open(answers_filename, 'r') as f:
        y_expected = np.array([read_target_from_vw(value) for value in f.readlines()])
        
    return r2_score(y_expected, y_pred)

In [68]:
%%local

! vw --help | head

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1
driver:
  --onethread           Disable parse thread
VW options:
  --ring_size arg (=256, ) size of example ring
  --strict_parse           throw on malformed examples
Update options:
  -l [ --learning_rate ] arg Set learning rate
  --power_t arg              t power value
  --decay_learning_rate arg  Set Decay factor for learning_rate between passes
  --initial_t arg            initial t value


Обучаем VW на одном файле целиком

In [130]:
%%local
%%time

! vw --final_regressor drugs.model.bin train.full.vw \
    --onethread \
    --learning_rate 20.0 \
    --bit_precision 23 \
    --passes 40 \
    --ngram r2 \
    --interactions dc \
    --cache -k

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
final_regressor = drugs.model.bin
Num weight bits = 23
learning rate = 20
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.full.vw.cache
Reading datafile = train.full.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
9.000000 9.000000            1            1.0   3.0000   0.0000      139
23.095423 37.190845            2            2.0   7.0000   0.9016      123
24.683209 26.270996            4            4.0   0.0000   6.5727      313
20.996466 17.309722            8            8.0  10.0000   2.1541      221
31.344119 41.691772           16           16.0   5.0000  13.8293      229
1672.860897 3314.377674           32           32.0   0.0000  84.7675      281
1124.977546 577.094196           64           64.0  20.0000  15.902

In [131]:
%%local

! vw --testonly --initial_regressor drugs.model.bin --predictions drugs.preductions.txt test.vw

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
only testing
predictions = drugs.preductions.txt
Num weight bits = 23
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.105660 0.105660            1            1.0   2.0000   1.6749      105
45.303623 90.501587            2            2.0   2.0000  11.5132      305
25.527859 5.752096            4            4.0   7.0000   5.1406       93
13.174219 0.820579            8            8.0  44.0000  43.9658      291
189.101929 365.029638           16           16.0  11.0000   7.2253       85
106.540190 23.978452           32           32.0  16.0000  12.7916      251
121.725979 136.911768           64           64.0  12.0000  11.4016      305
289.456672 457.187364          128

In [132]:
%%local

calc_r2('drugs.preductions.txt', 'test.vw')

0.6457895704437322

Обучили модель на **0.64** за **38** секунд.

Посмотрим, что будет если мы обучим модель только на части данных

In [99]:
%%local
%%time

! vw --final_regressor drugs.model.bin train.part1.vw \
    --onethread \
    --learning_rate 20.0 \
    --bit_precision 23 \
    --passes 40 \
    --ngram r2 \
    --interactions dc \
    --cache -k

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
final_regressor = drugs.model.bin
Num weight bits = 23
learning rate = 20
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.part1.vw.cache
Reading datafile = train.part1.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
9.000000 9.000000            1            1.0   3.0000   0.0000      139
23.095423 37.190845            2            2.0   7.0000   0.9016      123
24.683209 26.270996            4            4.0   0.0000   6.5727      313
20.996466 17.309722            8            8.0  10.0000   2.1541      221
31.344119 41.691772           16           16.0   5.0000  13.8293      229
1672.860897 3314.377674           32           32.0   0.0000  84.7675      281
1124.977546 577.094196           64           64.0  20.0000  15.9

In [100]:
%%local

! vw --testonly --initial_regressor drugs.model.bin --predictions drugs.preductions.txt test.vw

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
only testing
predictions = drugs.preductions.txt
Num weight bits = 23
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
4.000000 4.000000            1            1.0   2.0000   0.0000      105
4.000000 4.000000            2            2.0   2.0000   0.0000      305
10.160723 16.321446            4            4.0   7.0000   2.1376       93
19.146634 28.132545            8            8.0  44.0000  43.2707      291
223.987061 428.827488           16           16.0  11.0000  21.0392       85
164.388026 104.788992           32           32.0  16.0000   0.0000      251
155.765617 147.143208           64           64.0  12.0000  12.3560      305
270.828497 385.891376          12

In [101]:
%%local

calc_r2('drugs.preductions.txt', 'test.vw')

0.43776308727255231

Гораздо быстрее обучились, но потеряли в качестве. 

Модель на **0.43** за **9** секунд

**Мораль** - семплирование не самых удачный подход, чтобы получать качество, нужно засовывать в модель вообще все данные.

Запустим в фоновом режиме `spanning_tree` и проверим что он правда работает.

Далее воркеры будут подключаться к нему по tcp.

In [79]:
%%bash --bg --out OUT --err ERR
spanning_tree --nondaemon

Starting job # 0 in a separate thread.


In [102]:
%%local

! ps aux | grep spanning_tree

spark    11265  0.0  0.0  22276  4040 ?        S    09:55   0:00 spanning_tree --nondaemon
spark    26120  0.0  0.0  13296  2936 pts/2    Ss+  10:08   0:00 /bin/sh -c  ps aux | grep spanning_tree
spark    26126  0.0  0.0  15000  1032 pts/2    R+   10:08   0:00 grep spanning_tree


Пора запускать рабочих. Для этого используется уже известная команда vw, в которую просто добавляются специальные параметры

* `--span_server` - указываем адрес, где находится менеджер (spanning_tree). В нашем случае это localhost. В реальной жизни там мог бы быть IP адрес другой машины
* `--unique_id` - так как один spanning_tree может обрабатывать сразу много различных процессов обучения, то необходимо их как-то разграничить. Для этого используется unique_id - это число, которое должно быть одинаковым для всех ваших рабочих, чтобы их не перепутали с другими. Например ваш коллега также обучает VW но для другой задачи - он может подключить свои VW к этому же spanning_tree указав для них unique_id = 0. В таком случае вам, чтобы подключиться, нужно запускать свои рабочие например с unique_id = 5, чтобы они не смешались с рабочими вашего коллеги.
* `--total` - число рабочих, которое вы планируете подключить в текущей сессии обучения
--node - идентификатор текущего рабочего. Нумерация начинается с нуля, поэтому если вы хотите запустить 3 рабочих, то им нужно выдать значения для --node 0, 1 и 2.
* `-d` - данные для обработки для текущего рабочего
Все остальные параметры обучения должны быть одинаковыми для всех рабочих.

Чтобы сохранить коэффициенты полученной модели, необходимо для какого-то одного рабочего указать через `-f` или `--final_regressor` файл, куда записать результат. Точно также, как мы это делали в предыдущей лабораторной.

Запустим двух рабочих. Первого запустим также в фоне, а вот второй запустим прямо в ноутбуке и будем следить за процессом обучения.

In [133]:
%%bash --bg --out OUT --err ERR

vw -d train.part1.vw \
    --span_server localhost \
    --total 2 \
    --node 0 \
    --unique_id 1 \
    --learning_rate 20.0 \
    --bit_precision 23 \
    --passes 40 \
    --ngram r2 \
    --interactions dc \
    --cache -k

Starting job # 9 in a separate thread.


In [134]:
%%local 
! ps aux | grep vw

spark     3187  0.0  0.5 238320 147084 ?       Rl   10:48   0:00 vw -d train.part1.vw --span_server localhost --total 2 --node 0 --unique_id 1 --learning_rate 20.0 --bit_precision 23 --passes 40 --ngram r2 --interactions dc --cache -k
spark     3190  0.0  0.0  13296  3032 pts/2    Ss+  10:48   0:00 /bin/sh -c  ps aux | grep vw
spark     3194  0.0  0.0  15000   952 pts/2    R+   10:48   0:00 grep vw


In [135]:
%%local
%%time

! vw -d train.part2.vw \
    --span_server localhost \
    --total 2 \
    --node 1 \
    --unique_id 1 \
    --learning_rate 20.0 \
    --bit_precision 23 \
    --passes 40 \
    --ngram r2 \
    --interactions dc \
    --cache -k \
    -f drugs.model.bin

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
final_regressor = drugs.model.bin
Num weight bits = 23
learning rate = 20
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.part2.vw.cache
Reading datafile = train.part2.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0   0.0000   0.0000      291
24.500000 49.000000            2            2.0   7.0000   0.0000      325
14.530513 4.561025            4            4.0   2.0000   3.2344      191
8.172677 1.814841            8            8.0   4.0000   3.6167      321
33.730052 59.287428           16           16.0   0.0000  11.4410      257
133.395619 233.061186           32           32.0   9.0000  20.9062      107
459.865899 786.336180           64           64.0   0.0000   3.1751   

In [136]:
%%local

! vw --testonly --initial_regressor drugs.model.bin --predictions drugs.preductions.txt test.vw

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
only testing
predictions = drugs.preductions.txt
Num weight bits = 23
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
4.000000 4.000000            1            1.0   2.0000   0.0000      105
2.054122 0.108243            2            2.0   2.0000   2.3290      305
2.844955 3.635789            4            4.0   7.0000   6.3951       93
1.785510 0.726064            8            8.0  44.0000  44.6815      291
199.965630 398.145750           16           16.0  11.0000   7.5989       85
118.965405 37.965181           32           32.0  16.0000  13.7154      251
124.239531 129.513657           64           64.0  12.0000  11.7332      305
259.078211 393.916891          128    

In [137]:
%%local

calc_r2('drugs.preductions.txt', 'test.vw')

0.65761500207159129

Качество получилось даже немного больше, чем при одиночном запуске.

Сильного ускорения по времени мы не увидели, потому что мы все это запускаем на одной машине. Однако если запускать эти воркеры на разных машинах и на больших объемах данных, то можно увидеть сильное ускорение процесса обучения.

И основное достижение этого алгоритма - теперь мы можем размещать данные по нескольким машинам, что позволяет нам теоретически обработать датасет произвольного размера.

### VW на Hadoop

VW достаточно несложно запустить в виде обычной MapReduce задачи. Для этого даже есть готовый скрипт, который написан авторами инструмента. 

Почитать про то, как запускать этот инструмент на Hadoop можно вот здесь - https://github.com/VowpalWabbit/vowpal_wabbit/tree/master/cluster .

Мы же с вами более внимательно рассмотрим более удобный интерфейс для распределенного обучения VW на кластере.

### MMLSpark

Существует целый набор библиотек для Spark от Microsoft, который позволяет удобно и быстро запускать распределенные алгоритмы на кластере Spark. Про все возможности можно почитать на официальном GitHub - https://github.com/Azure/mmlspark .

Мы с вами воспользуемся двумя инструментами оттуда - VW и LightGBM (градиентный бустинг).

Чтобы поставить mmlspark в окружение с lyvi (это окржуение присутствует в кластере azure), достаточно просто переконфигурировать сессию спарка.

In [138]:
%%configure -f
{
    "name": "mmlspark",
    "conf": {
        "spark.jars.packages": "com.microsoft.ml.spark:mmlspark_2.11:1.0.0-rc3",
        "spark.jars.repositories": "https://mmlspark.azureedge.net/maven",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11,org.scalactic:scalactic_2.11,org.scalatest:scalatest_2.11"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1613979686616_0008,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1613979686616_0008,pyspark,idle,Link,Link,✔


In [177]:
from pyspark.sql.functions import when, col
from pyspark.ml import Pipeline
from mmlspark.vw import VowpalWabbitFeaturizer, VowpalWabbitRegressor

In [178]:
data = spark.read.option("delimiter", "\t").csv('/drugs/data/*', header=True, inferSchema=True)

In [179]:
train, test = (
    data
    .na.drop('any')
    .randomSplit([0.9, 0.1], 422)
)

In [180]:
data.show()

+--------------------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|                 _c0|            drugName|           condition|              review|rating|             date|usefulCount|
+--------------------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|              206461|           Valsartan|Left Ventricular ...|"""It has no side...|   9.0|     May 20, 2012|         27|
|               95260|          Guanfacine|                ADHD|"""My son is half...|  null|             null|       null|
|We have tried man...|                 8.0|      April 27, 2010|                 192|  null|             null|       null|
|               92703|              Lybrel|       Birth Control|"""I used to take...|  null|             null|       null|
|The positive side...|                 5.0|   December 14, 2009|                  17|  null|             null|       null|
|              1

Создадим объект для создания признаков в формате VW. Он принимает dataframe и возвращает dataframe но уже с новой колонкой, в которой записаны эти признаки

In [181]:
vw_featurizer = VowpalWabbitFeaturizer(
    inputCols=["drugName", "condition", "rating"], 
    stringSplitInputCols=["review"],
    outputCol="features",
    numBits=24
)

In [182]:
x = vw_featurizer.transform(train).rdd.first()
x['features']

SparseVector(16777216, {242607: 1.0, 322542: 1.0, 554393: 1.0, 667916: 1.0, 767305: 1.0, 1227143: 1.0, 1652180: 1.0, 1988378: 1.0, 2076500: 1.0, 2711816: 1.0, 3213474: 1.0, 3282905: 1.0, 3329565: 3.0, 3449553: 1.0, 3586426: 3.0, 3702762: 1.0, 4764873: 1.0, 4883620: 1.0, 5316217: 1.0, 5366447: 1.0, 5886655: 1.0, 6404762: 1.0, 6820536: 2.0, 7166710: 1.0, 7296596: 1.0, 7709679: 1.0, 9418434: 1.0, 9481902: 1.0, 9504901: 1.0, 9829382: 1.0, 11227913: 1.0, 11655066: 1.0, 11814010: 1.0, 12095292: 1.0, 12501039: 1.0, 13034204: 1.0, 13096767: 1.0, 13413959: 1.0, 13732862: 1.0, 13990054: 1.0, 14295339: 1.0, 14599339: 1.0, 15072472: 1.0, 15177082: 1.0, 15477096: 1.0, 16165677: 1.0, 16588573: 2.0})

Создадим объект для обучения классификатора. Схема работы точно такая же - принимает на вход dataframe и потом может модифицировать другой dataframe, делая предсказание.

In [183]:
args = "--learning_rate 20.0 --bit_precision 23 --ngram r2 --interactions dc"
vw_model = VowpalWabbitRegressor(
    featuresCol="features",
    labelCol="usefulCount",
    args=args,
    numPasses=40
)

Соберем их в единый пайплайн

In [184]:
vw_pipeline = Pipeline(stages=[vw_featurizer, vw_model])

In [185]:
vw_trained = vw_pipeline.fit(train)

In [186]:
prediction = vw_trained.transform(test)

In [187]:
prediction.show()

+------+--------------------+----------------+--------------------+------+------------------+-----------+--------------------+------------------+------------------+
|   _c0|            drugName|       condition|              review|rating|              date|usefulCount|            features|     rawPrediction|        prediction|
+------+--------------------+----------------+--------------------+------+------------------+-----------+--------------------+------------------+------------------+
| 10000|      Lo Loestrin Fe|   Birth Control|"""I was on this ...|   7.0|    April 10, 2013|          4|(16777216,[242607...|               0.0|               0.0|
| 10007|      Lo Loestrin Fe|   Birth Control|"""I posted on th...|   9.0|    March 10, 2013|         18|(16777216,[463906...|21.216596603393555|21.216596603393555|
|100105|Desogestrel / eth...|   Birth Control|"""For the most p...|   8.0| November 30, 2016|          3|(16777216,[176662...| 2.666006088256836| 2.666006088256836|
|100157|De

In [188]:
from mmlspark.train import ComputeModelStatistics
metrics = ComputeModelStatistics(
    evaluationMetric='regression',
    labelCol='usefulCount',
    scoresCol='prediction'
).transform(prediction)

In [189]:
metrics.show()

+------------------+-----------------------+-------------------+-------------------+
|mean_squared_error|root_mean_squared_error|                R^2|mean_absolute_error|
+------------------+-----------------------+-------------------+-------------------+
| 675.2070439236329|      25.98474637020021|0.46103020070572054| 16.375469583477777|
+------------------+-----------------------+-------------------+-------------------+

### SparkML

Нужно отметить, что в стандартной библиотеке Spark присутствует модуль для машинного обучения.

**ОДНАКО** нужно сказать, что работает он крайне плохо. Лучшее, что вы можете с ним сделать - это попробовать один раз его запустить и понять, что больше никогда не будете его использовать.

Это правда важно, потому что это не звучит слишком убедительно, что стандартная библиотека для ML насколько уж плохо работет и наверное все таки есть случаи, когда она работает хорошо, правда ведь? Ответ - вполне возможно. Чтобы вам самим понять, есть ли такие случаи, попробуйте самостоятельно что-то обучить на SparkML и прочувствуйте границы применимости :)

In [19]:
%%configure -f
{"executorMemory": "20000M", "executorCores": 4, "numExecutors":5}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1614006317960_0008,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1614006317960_0008,pyspark,idle,Link,Link,✔


In [20]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderEstimator, StringIndexer, VectorAssembler

In [34]:
data = spark.read.option("delimiter", "\t").csv('/drugs/data/*', header=True, inferSchema=True)

In [35]:
data = (
    data
    .na.drop('any')
    .withColumn('ratingNum', data.rating.cast('integer'))
)


train, test = data.randomSplit([0.9, 0.1], 422)
train, test = train.cache(), test.cache()

In [19]:
tokenizer = Tokenizer(inputCol="review", outputCol="words")
wordsData = tokenizer.transform(train)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2**23)
featurizedData = hashingTF.transform(wordsData)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

rescaledData = idfModel.transform(featurizedData)

In [20]:
rescaledData.limit(1).show()

+------+--------------------+-------------+--------------------+------+-------------+-----------+---------+--------------------+--------------------+--------------------+
|   _c0|            drugName|    condition|              review|rating|         date|usefulCount|ratingNum|               words|         rawFeatures|            features|
+------+--------------------+-------------+--------------------+------+-------------+-----------+---------+--------------------+--------------------+--------------------+
|100002|Desogestrel / eth...|Birth Control|"""I have been ta...|   4.0|July 12, 2017|          2|        4|["""i, have, been...|(8388608,[15664,5...|(8388608,[15664,5...|
+------+--------------------+-------------+--------------------+------+-------------+-----------+---------+--------------------+--------------------+--------------------+

In [21]:
stringIndexer = StringIndexer(inputCol='drugName', outputCol = "drugIndex").setHandleInvalid("skip")
encoder = OneHotEncoder(inputCol="drugIndex", outputCol="drugVec")

pipeline = Pipeline(stages=[stringIndexer, encoder])
ohe = pipeline.fit(rescaledData).transform(rescaledData)

In [22]:
x = ohe.limit(1).rdd.first()
x

Row(_c0='100002', drugName='Desogestrel / ethinyl estradiol', condition='Birth Control', review='"""I have been taking this birth control for four months now. The first was the worst, I was irritable, frequently sad, lethargic and just overall moody. It cleared up my face and didn&#039;t change much in the way of flow on my period but, did make them shorter. My sex drive has also decreased as well as my eating habits. I find myself less hungry but, my breasts have grown in size. Now four months later I&#039;ve been spotting non-stop for the past two weeks and my face is breaking out in acne that rests under the skin. Going to change my prescription soon but, for awhile it was definitely tolerable."""', rating='4.0', date='July 12, 2017', usefulCount=2, ratingNum=4, words=['"""i', 'have', 'been', 'taking', 'this', 'birth', 'control', 'for', 'four', 'months', 'now.', 'the', 'first', 'was', 'the', 'worst,', 'i', 'was', 'irritable,', 'frequently', 'sad,', 'lethargic', 'and', 'just', 'overa

In [24]:
x['drugVec']

SparseVector(3488, {63: 1.0})

Подготавливаем признаки

In [25]:
wordsData = tokenizer.transform(train)

tokenizer = Tokenizer(inputCol="review", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2**23)
idf = IDF(inputCol="rawFeatures", outputCol="revviewFeatures")

stringIndexerCondition = StringIndexer(inputCol='condition', outputCol = "conditionIndex").setHandleInvalid("skip")
encoderCondition = OneHotEncoder(inputCol="conditionIndex", outputCol="conditionVec")

stringIndexerDrug = StringIndexer(inputCol='drugName', outputCol = "drugIndex").setHandleInvalid("skip")
encoderDrug = OneHotEncoder(inputCol="drugIndex", outputCol="drugVec")

assembler = VectorAssembler(inputCols=["drugVec", "conditionVec", "revviewFeatures", 'ratingNum'], outputCol="features")

preproc = Pipeline(stages=[
    tokenizer,
    hashingTF,
    idf,
    stringIndexerCondition,
    encoderCondition,
    stringIndexerDrug,
    encoderDrug,
    assembler
])

In [ ]:
train_proc = preproc.fit(train).transform(train).cache()

In [ ]:
train_proc.show()

In [5]:
tokenizer = Tokenizer(inputCol="review", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2**23)
idf = IDF(inputCol="rawFeatures", outputCol="revviewFeatures")

preproc = Pipeline(stages=[
    tokenizer,
    hashingTF,
    idf
])

Запустить сбор признаков, который написан, у вас скорее всего не получится. Поэтому попробуем урезать количество вычислений - может быть получится.

In [36]:
stringIndexerCondition = StringIndexer(inputCol='condition', outputCol = "conditionIndex").setHandleInvalid("skip")
encoderCondition = OneHotEncoder(inputCol="conditionIndex", outputCol="conditionVec")

stringIndexerDrug = StringIndexer(inputCol='drugName', outputCol = "drugIndex").setHandleInvalid("skip")
encoderDrug = OneHotEncoder(inputCol="drugIndex", outputCol="drugVec")

assembler = VectorAssembler(inputCols=["drugVec", "conditionVec", 'ratingNum'], outputCol="features")


preproc = Pipeline(stages=[
    stringIndexerCondition,
    encoderCondition,
    stringIndexerDrug,
    encoderDrug,
    assembler
])

In [37]:
preproc = preproc.fit(data)

In [38]:
train_proc = preproc.transform(train).cache()
test_proc = preproc.transform(test).cache()

In [39]:
train_proc.show()

+------+--------------------+-------------+--------------------+------+------------------+-----------+---------+--------------+---------------+---------+-----------------+--------------------+
|   _c0|            drugName|    condition|              review|rating|              date|usefulCount|ratingNum|conditionIndex|   conditionVec|drugIndex|          drugVec|            features|
+------+--------------------+-------------+--------------------+------+------------------+-----------+---------+--------------+---------------+---------+-----------------+--------------------+
|100029|Desogestrel / eth...|Birth Control|"""I was switched...|   5.0|  December 6, 2017|          0|        5|           0.0|(896,[0],[1.0])|     60.0|(3572,[60],[1.0])|(4469,[60,3572,44...|
|100071|Desogestrel / eth...|Birth Control|"""So I&#039;ve o...|   8.0|     March 7, 2017|          7|        8|           0.0|(896,[0],[1.0])|     60.0|(3572,[60],[1.0])|(4469,[60,3572,44...|
|100087|Desogestrel / eth...|Birth 

Если все таки удалось собрать датасет, то запускаем линейную регрессию

In [40]:
lr = LinearRegression(featuresCol='features', labelCol='usefulCount', maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [41]:
lrModel = lr.fit(train_proc)

In [42]:
lrModel.coefficients

SparseVector(4469, {0: -0.6743, 6: -0.0783, 7: 0.2372, 8: -2.602, 11: 18.7913, 12: -2.539, 13: 7.4739, 15: -4.8054, 17: 0.4586, 18: 1.5943, 19: -4.8051, 20: -1.6743, 22: -1.0804, 24: 1.6261, 25: 5.4599, 27: -3.7968, 28: 15.3822, 30: -1.9413, 31: -6.3131, 32: 4.7946, 34: -1.0008, 36: 17.4721, 37: -1.6746, 41: -0.9766, 43: 8.391, 44: 16.0544, 45: 8.4384, 46: -4.1876, 47: 19.8013, 49: 1.4817, 51: 21.742, 54: -2.4582, 59: 2.7534, 61: -6.7088, 63: 16.3311, 64: 3.8718, 68: -8.194, 69: 3.5768, 70: -0.0624, 73: 2.2912, 74: 20.8008, 76: 8.7944, 77: 12.2401, 83: 10.5261, 85: 0.339, 88: -2.3094, 89: -1.2361, 91: 3.2148, 94: -4.7645, 97: 10.7624, 101: 5.7298, 104: 8.1481, 106: 14.3783, 107: 0.4101, 108: -1.2586, 109: 12.4, 110: 20.6213, 111: 8.2038, 113: -5.6766, 115: 1.6724, 118: 15.4201, 119: 2.6892, 120: 5.7309, 122: 7.8388, 123: 10.2542, 124: 27.0023, 126: -0.6129, 132: 0.6256, 139: -0.5122, 140: 1.4564, 142: -0.513, 143: 39.0297, 145: 15.5268, 147: 17.2809, 149: 7.4676, 152: 4.0548, 153: 6.80

In [43]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = lrModel.transform(test_proc)

In [44]:
predictions.show()

+------+--------------------+--------------------+--------------------+------+------------------+-----------+---------+--------------+-----------------+---------+-------------------+--------------------+------------------+
|   _c0|            drugName|           condition|              review|rating|              date|usefulCount|ratingNum|conditionIndex|     conditionVec|drugIndex|            drugVec|            features|        prediction|
+------+--------------------+--------------------+--------------------+------+------------------+-----------+---------+--------------+-----------------+---------+-------------------+--------------------+------------------+
|100012|Desogestrel / eth...|       Birth Control|"""I&#039;ve been...|   9.0|     June 17, 2017|          2|        9|           0.0|  (896,[0],[1.0])|     60.0|  (3572,[60],[1.0])|(4469,[60,3572,44...|14.531704383354949|
|100123|Desogestrel / eth...|       Birth Control|"""I haven&#039;t...|   1.0|  November 2, 2016|          2

In [45]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="usefulCount", metricName="r2")
lr_evaluator.evaluate(predictions)

0.2824947550170618

Далее будем говорить уже про деревья решений. Там мы будем смотреть на другой полезный инструмент, который называется H2O.

Для удобства, я разнес эти части по разным ноутбукам. Переходите в следующий ноутбук **7.2**